 # 1. primera metodologia

### 1.0 crear datos sinteticos

In [1]:
import pandas as pd
import numpy as np

#paso 0: parametros
n_activos = 5
periodos=200
n_portafolios = 48

# Paso 1: Crear el DataFrame de retornos (5 activos)
fechas = pd.date_range('1974-01-02', periods=periodos, freq='B')  # 20 días hábiles
np.random.seed(42)
data = np.random.randn(periodos, n_activos)  # Generación de retornos aleatorios
df = pd.DataFrame(data, index=fechas, columns=[f'Activo_{i+1}' for i in range(n_activos)])
df


,Activo_1,Activo_2,Activo_3,Activo_4,Activo_5
1974-01-02,0.496714,-0.138264,0.647689,1.523030,-0.234153
1974-01-03,-0.234137,1.579213,0.767435,-0.469474,0.542560
1974-01-04,-0.463418,-0.465730,0.241962,-1.913280,-1.724918
1974-01-07,-0.562288,-1.012831,0.314247,-0.908024,-1.412304
1974-01-08,1.465649,-0.225776,0.067528,-1.424748,-0.544383
...,...,...,...,...,...
1974-10-02,-1.889541,-0.452306,-2.423879,-1.583903,0.760415
1974-10-03,0.785800,0.425458,-0.966976,-0.047711,-0.003603
1974-10-04,-1.158365,1.503398,0.877362,-0.220964,0.026886
1974-10-07,0.208383,-2.041735,-0.247177,-0.681984,-1.001620


In [2]:
# Paso 2: Crear portafolios con pesos aleatorios para 48 portafolios (48 combinaciones)
  # Número de portafolios
pesos = np.random.rand(n_portafolios, n_activos)  # 48 portafolios, 5 activos
pesos = pesos / pesos.sum(axis=1)[:, np.newaxis]  # Normalizamos para que sumen 1

# Multiplicamos los retornos por los pesos para obtener los portafolios
portafolios = np.dot(df.values, pesos.T)

# Paso 3: Crear un DataFrame para los portafolios con los valores calculados
df_portafolios = pd.DataFrame(portafolios, index=fechas, columns=[f'Portafolio_{i+1}' for i in range(n_portafolios)])
df_portafolios


,Portafolio_1,Portafolio_2,Portafolio_3,Portafolio_4,Portafolio_5,Portafolio_6,Portafolio_7,Portafolio_8,Portafolio_9,Portafolio_10,...,Portafolio_39,Portafolio_40,Portafolio_41,Portafolio_42,Portafolio_43,Portafolio_44,Portafolio_45,Portafolio_46,Portafolio_47,Portafolio_48
1974-01-02,0.941081,0.541698,0.613026,0.428033,0.150113,0.588335,0.386217,0.645351,0.030778,0.499727,...,0.452376,0.495528,0.591507,0.579141,0.479675,0.281129,0.631395,0.545081,0.608903,0.460064
1974-01-03,0.182081,0.134410,0.371005,0.422966,0.728846,0.480728,0.662202,0.238773,0.974221,0.527886,...,0.564040,0.209178,0.265252,0.140348,0.380464,0.668534,0.378534,0.152830,0.423318,0.500928
1974-01-04,-0.837204,-1.024621,-0.843339,-0.729371,-0.671868,-1.121651,-0.821116,-0.892498,-0.721543,-0.805194,...,-0.776251,-1.320198,-1.039730,-0.964126,-0.980757,-0.612604,-0.913813,-0.912989,-0.734473,-0.727034
1974-01-07,-0.416748,-0.734159,-0.712635,-0.631950,-0.859158,-0.883899,-0.719781,-0.595746,-0.943501,-0.677673,...,-0.714519,-0.981158,-0.778668,-0.674829,-0.763242,-0.820115,-0.765047,-0.776251,-0.511341,-0.717017
1974-01-08,-0.460909,-0.028106,-0.114908,0.073981,0.237604,-0.593746,-0.327496,-0.188580,-0.032407,-0.235160,...,-0.147118,-0.371567,-0.240285,-0.024139,-0.230770,0.292822,-0.212853,0.218290,-0.238357,0.044833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1974-10-02,-1.822827,-1.146593,-1.368768,-1.235708,-0.868881,-0.981789,-0.992786,-1.380385,-0.574311,-1.203331,...,-1.170135,-0.736707,-1.150529,-1.257221,-1.031776,-1.127915,-1.311928,-1.288582,-1.443213,-1.265055
1974-10-03,-0.287352,0.063482,0.194468,0.036333,0.389905,0.092275,-0.068416,-0.098046,0.301542,-0.002226,...,0.096571,0.148052,0.096193,0.023319,0.016873,0.498009,0.220067,0.357585,-0.199835,0.234477
1974-10-04,0.222481,-0.213760,0.181103,0.125506,0.336979,0.433653,0.543592,0.052281,0.689355,0.388930,...,0.385333,-0.072912,0.047667,-0.178702,0.156496,0.304780,0.219893,-0.265366,0.323189,0.243967
1974-10-07,-0.515840,-0.450478,-0.802835,-0.621851,-1.027795,-1.093845,-0.981060,-0.526467,-1.325574,-0.864410,...,-0.912733,-0.763438,-0.709907,-0.428042,-0.743974,-1.027258,-0.882973,-0.533685,-0.643929,-0.846322


### 1.1 Ortogonalización

In [3]:

# Paso 4: Calcular la matriz de covarianzas con una ventana móvil
ventana = 121  # Tamaño de la ventana móvil (ajustado para que funcione con los 20 días)
cov_matrices = []
for i in range(ventana, len(df_portafolios)):
    ventana_datos = df_portafolios.iloc[i-ventana:i]
    cov_matrices.append(ventana_datos.cov().values)



In [5]:
# lo hacemos para todos los t
# Procesar cada matriz de covarianzas
portafolios_dinamicos= []  # Lista para almacenar DataFrames de portafolios para cada matriz

for t, cov in enumerate(cov_matrices):
    cov_matrix = cov

    # Paso 5: Descomponer la matriz de covarianzas (autovalores y autovectores)
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

    # Paso 6: Ordenar los autovalores y autovectores en orden descendente
    idx = eigenvalues.argsort()[::-1]  # Índices ordenados
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]

    # Paso 7: Normalizar los eigenvectores seleccionados
    eigenvectors_normalized = eigenvectors / np.linalg.norm(eigenvectors, axis=0)

    # Paso 8: Crear portafolios ortogonales a partir de los eigenvectores normalizados
    df_ortogonales = pd.DataFrame(
        np.dot(df_portafolios.values, eigenvectors_normalized),
        index=fechas,
        columns=[f'portafolio_ortogonal_{i+1}' for i in range(eigenvectors_normalized.shape[1])]
    )
    
    # Paso 9: Determinar dinámicamente el número de eigenportafolios principales (l)
    # Regla: l = 1 + {median(λ) < λ_i}
    #l=3 o 5 (Famma and French, 1993)
    mediana_eigenvalores = np.median(eigenvalues)  # Calcular la mediana de los eigenvalores pq factor might be time varying
    l = sum(eigenvalues > mediana_eigenvalores)  # Contar cuántos eigenvalores superan la mediana

    # Paso 10: Seleccionar los primeros l portafolios ortogonales
    df_ortogonales_l = df_ortogonales.iloc[:, :l]
    portafolios_dinamicos.append(df_ortogonales_l)
    #print(f"Número de eigenportafolios seleccionados dinámicamente l={l} en t={t}")
    #print(df_ortogonales_l.head())


In [6]:
# siempre es l=24, en esencia no cambia

In [7]:
# Paso 11: Seleccionar los primeros l portafolios ortogonales
portafolios_dinamicos[1]

,portafolio_ortogonal_1,portafolio_ortogonal_2,portafolio_ortogonal_3,portafolio_ortogonal_4,portafolio_ortogonal_5,portafolio_ortogonal_6,portafolio_ortogonal_7,portafolio_ortogonal_8,portafolio_ortogonal_9,portafolio_ortogonal_10,...,portafolio_ortogonal_15,portafolio_ortogonal_16,portafolio_ortogonal_17,portafolio_ortogonal_18,portafolio_ortogonal_19,portafolio_ortogonal_20,portafolio_ortogonal_21,portafolio_ortogonal_22,portafolio_ortogonal_23,portafolio_ortogonal_24
1974-01-02,-3.346768,1.294686,0.504514,0.103496,-0.283108,7.376551e-16,1.713364e-15,1.768412e-15,4.372831e-16,2.776855e-16,...,2.975306e-16,-1.246442e-16,4.486799e-16,-1.772971e-16,1.429593e-16,2.822963e-16,-1.068184e-16,1.418037e-16,-4.536984e-16,2.385041e-16
1974-01-03,-3.022495,-1.119619,0.101935,-0.041975,0.658141,-2.849869e-16,-7.527556e-16,1.375421e-17,-3.822555e-16,-2.130120e-16,...,-1.406669e-16,2.912667e-17,-5.241682e-17,-2.167366e-16,3.703177e-17,-3.020695e-16,1.003705e-16,5.123915e-17,1.298951e-16,-3.311282e-16
1974-01-04,6.221546,-0.365620,0.718215,-0.591964,0.995676,-1.084063e-15,-3.441702e-15,-2.339152e-15,-5.129897e-16,-5.287527e-16,...,-1.873330e-16,6.496979e-17,-8.579828e-18,5.357862e-16,-1.492517e-16,2.883294e-16,2.337212e-16,-3.083863e-17,7.879391e-16,-9.091913e-18
1974-01-07,5.100495,0.609401,0.399097,-0.241145,0.709580,-1.674603e-15,-2.372318e-15,-1.498624e-15,-8.164118e-16,-1.255509e-16,...,-1.483822e-16,2.190127e-16,-8.914302e-18,4.286413e-16,-1.444101e-16,6.997710e-17,9.543874e-17,1.436681e-16,4.682131e-16,2.735700e-16
1974-01-08,1.336078,-0.621745,0.437522,-1.455056,-0.098887,-2.035024e-17,9.431595e-17,-1.800883e-16,-6.150203e-16,-1.647782e-16,...,1.342091e-16,-1.699886e-16,6.063710e-16,1.471094e-16,1.069237e-16,5.229936e-16,2.152796e-16,5.756344e-17,-1.325752e-16,1.100065e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1974-10-02,7.684381,-1.758051,-1.515455,1.091420,-0.131064,-6.113806e-16,-2.460997e-15,-3.563739e-15,-3.928264e-16,-3.854701e-16,...,-6.026989e-16,6.367534e-16,-9.456179e-16,7.012168e-16,8.116413e-16,-7.489861e-16,1.415658e-16,-5.151030e-16,3.187805e-16,-1.903058e-16
1974-10-03,-0.349070,-0.856395,0.339253,-0.154245,-0.728520,2.434540e-16,7.158239e-16,5.159666e-17,-7.997334e-17,3.531251e-17,...,8.232601e-17,4.873473e-17,1.982394e-16,8.229393e-17,2.032351e-16,2.271662e-16,-2.158568e-17,-2.437965e-16,-3.421691e-16,-1.366450e-16
1974-10-04,-1.600778,-0.743737,0.294534,0.556193,1.059243,-9.142636e-16,-1.907110e-15,-5.300025e-16,8.493134e-17,-2.182028e-16,...,-3.257457e-16,-2.980479e-17,-4.074181e-16,-1.588549e-17,2.659290e-17,-5.512528e-16,2.392544e-16,7.940863e-18,4.939995e-16,-3.504629e-16
1974-10-07,5.508225,1.121288,-0.126041,-0.592547,-0.059786,-8.341383e-17,-6.566799e-16,-9.219727e-16,-1.521686e-16,-9.229457e-17,...,-8.594754e-17,-1.751017e-17,1.726420e-16,3.087004e-16,-2.580418e-16,4.257102e-16,-1.526862e-17,-1.696728e-16,7.068675e-17,4.107415e-16


In [8]:
portafolios_dinamicos[2]

,portafolio_ortogonal_1,portafolio_ortogonal_2,portafolio_ortogonal_3,portafolio_ortogonal_4,portafolio_ortogonal_5,portafolio_ortogonal_6,portafolio_ortogonal_7,portafolio_ortogonal_8,portafolio_ortogonal_9,portafolio_ortogonal_10,...,portafolio_ortogonal_15,portafolio_ortogonal_16,portafolio_ortogonal_17,portafolio_ortogonal_18,portafolio_ortogonal_19,portafolio_ortogonal_20,portafolio_ortogonal_21,portafolio_ortogonal_22,portafolio_ortogonal_23,portafolio_ortogonal_24
1974-01-02,-3.346265,1.340029,0.374014,0.067027,-0.291412,-1.354121e-15,5.369651e-16,-2.170110e-16,2.733080e-16,-1.457072e-16,...,9.936490e-17,-3.456782e-17,1.984533e-16,-1.867867e-18,1.389694e-16,-1.132153e-16,-7.990803e-17,1.167828e-16,4.998645e-17,-5.272376e-17
1974-01-03,-3.019222,-1.095545,0.242771,-0.024790,0.677387,7.831946e-16,-8.097721e-16,-2.275292e-17,-1.741706e-17,1.692938e-16,...,-3.001321e-16,-1.755601e-17,2.854386e-17,2.901002e-16,-7.583213e-17,1.328049e-16,-1.753997e-16,-3.419202e-16,-2.892585e-17,-2.604469e-17
1974-01-04,6.226378,-0.275707,0.692776,-0.627316,0.990792,4.643070e-16,3.978225e-17,2.719012e-16,-3.168838e-16,5.017009e-16,...,-9.108380e-17,1.331534e-16,-2.727894e-16,-1.945938e-16,-5.607953e-16,-2.573016e-16,-2.113068e-16,-4.711095e-16,1.018597e-16,3.760643e-17
1974-01-07,5.102620,0.658170,0.287172,-0.271813,0.693597,1.157488e-16,1.904839e-16,3.327160e-17,-1.699546e-16,1.368414e-16,...,-1.472213e-16,-2.847137e-17,4.679836e-17,-3.885078e-16,-4.376454e-16,-2.283215e-16,-1.153322e-17,-1.225075e-16,-1.194618e-17,1.603754e-16
1974-01-08,1.335964,-0.592373,0.427531,-1.471201,-0.084502,-8.301636e-16,2.868919e-16,1.859961e-16,-2.992836e-17,3.289867e-16,...,2.475983e-16,4.157819e-18,-5.495349e-17,3.368359e-16,-2.393267e-16,-9.928384e-17,2.635996e-17,-3.957849e-16,5.687863e-17,-2.174623e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1974-10-02,7.680868,-1.897612,-1.286859,1.172415,-0.134936,2.987769e-15,-1.040242e-15,5.815587e-16,-4.396724e-16,-3.886791e-17,...,-1.587366e-16,3.553032e-16,-6.352540e-16,-2.037105e-16,-2.244043e-16,6.241466e-16,1.099022e-16,-1.557720e-16,-3.852093e-16,1.080719e-16
1974-10-03,-0.348138,-0.822885,0.430093,-0.165963,-0.716955,-1.522026e-16,1.808240e-17,2.219356e-17,1.064903e-17,5.743140e-18,...,1.977977e-16,-1.684999e-16,1.768284e-17,3.072394e-16,6.203905e-17,6.395986e-17,-2.014038e-18,1.492080e-16,-1.266065e-16,-1.583549e-16
1974-10-04,-1.595055,-0.684587,0.412254,0.558863,1.067047,1.270238e-15,-9.137562e-16,-1.721323e-17,-4.250645e-17,1.470102e-16,...,-3.905873e-16,4.123337e-17,2.324053e-17,-2.447651e-16,-2.122710e-16,3.406131e-17,-1.472827e-16,-1.024124e-16,7.278449e-17,2.140072e-16
1974-10-07,5.504813,1.092089,-0.311317,-0.607839,-0.079902,-7.909853e-16,7.229806e-16,1.323684e-16,-2.582522e-16,1.445057e-16,...,2.705416e-16,9.497693e-17,-1.983099e-16,-8.542213e-17,-1.206653e-16,-2.151738e-16,1.811829e-16,5.137339e-17,1.526225e-17,4.480749e-17


## ahora que :( o sea tengo 79 matrices en portafolios dinamicos, se supone que cada uno es una realizacion de la verdadera distribucion??

In [13]:
df_portafolios=portafolios_dinamicos[-1] ######

### 1.2 ADTS

In [106]:
from scipy.stats import skew, kurtosis, norm
import numpy as np
import pandas as pd

# Suposición: df_portafolios y portafolios_dinamicos están definidos previamente

# Paso 11: Iterar sobre cada t para calcular Sharpe Ratios, límites superiores y recompensas
played_times = np.ones(len(df_portafolios.columns))  # Veces que cada portafolio ha sido jugado
success = np.ones(4)  # Éxitos iniciales para 4 estrategias
fail = np.ones(4)  # Fracasos iniciales para 4 estrategias
rewards = []  # Almacenar recompensas obtenidas
mv_rewards = []  # Almacenar recompensas para la estrategia de mínima varianza

# Iteración sobre las fechas o ventanas de tiempo
for t, df_l in enumerate(portafolios_dinamicos):
    # Paso 12: Calcular las varianzas de los portafolios ortogonales seleccionados
    portfolio_returns = df_l.values.T  # Retornos de los portafolios seleccionados
    cov_matrix_l = np.cov(portfolio_returns)  # Matriz de covarianza de los portafolios seleccionados
    portfolio_variances = np.diagonal(cov_matrix_l)  # Varianzas de los portafolios seleccionados

    # Paso 13: Calcular el Sharpe Ratio para cada portafolio
    sharpe_ratios = np.mean(portfolio_returns, axis=1) / np.sqrt(portfolio_variances)

    # Paso 14: Calcular el límite superior del Sharpe Ratio
    sharpe_ratio_upper_bound = sharpe_ratios + np.sqrt(
        (2 * np.log(t + 1)) / (1 + played_times[:len(sharpe_ratios)])
    )

    # Paso 15: Seleccionar los portafolios activos y pasivos óptimos
    l = len(df_l.columns)
    
    # Verificar que sharpe_ratio_upper_bound no esté vacío
    if len(sharpe_ratio_upper_bound) > 0:
        # Verificar que l no sea mayor que la longitud de sharpe_ratio_upper_bound
        if l < len(sharpe_ratio_upper_bound):
            action1 = np.argmax(sharpe_ratio_upper_bound[:l])  # Portafolio pasivo
            action2 = np.argmax(sharpe_ratio_upper_bound[l:]) + l  # Portafolio activo
        else:
            # Si l es mayor que el tamaño de sharpe_ratio_upper_bound, se ajusta
            action1 = np.argmax(sharpe_ratio_upper_bound)
            action2 = None
    else:
        # Manejar el caso cuando sharpe_ratio_upper_bound está vacío
        action1 = None
        action2 = None
    
    # Paso 16: Calcular los pesos óptimos entre los portafolios seleccionados
    if action1 is not None and action2 is not None:
        theta = portfolio_variances[action1] / (portfolio_variances[action1] + portfolio_variances[action2])
        # Estrategia 1: Pesos óptimos entre los portafolios seleccionados
        optimal_weight = (1 - theta) * eigenvectors_normalized[:, action1] + theta * eigenvectors_normalized[:, action2]
    else:
        # Si no se encuentran dos portafolios válidos, asignar un peso predeterminado
        optimal_weight = np.zeros(len(df_l.columns))  # Peso cero como valor predeterminado

    # Paso 17: Crear estrategias alternativas
    # Estrategia 2: Pesos iguales
    equal_weight = np.ones(len(cov_matrix_l)) / len(cov_matrix_l)

    # Estrategia 3: Mínima varianza
    inv_cov = np.linalg.inv(cov_matrix_l)
    min_var_weight = (inv_cov @ np.ones(len(cov_matrix_l)).reshape(-1, 1)) / (
        np.ones(len(cov_matrix_l)).reshape(-1, 1).T @ inv_cov @ np.ones(len(cov_matrix_l)).reshape(-1, 1)
    )

    # Estrategia 4: Basada en PSR ajustado
    psr_set = []
    for i in range(len(sharpe_ratios)):
        sr = sharpe_ratios[i]
        n = (1 + played_times[i]) / len(portfolio_returns[i])
        skewness = skew(portfolio_returns[i, :])
        kurt = kurtosis(portfolio_returns[i, :])
        nomin = (sr - np.mean(sharpe_ratios)) * np.sqrt(n)
        denom = np.sqrt(np.abs(1 + 0.5 * sr**2 - skewness * sr + ((kurt - 3) / 4) * sr**2) / (n - 1))
        psr_set.append(nomin / denom)

    psr_set = np.array([norm.cdf(x) for x in psr_set])
    sharpe_ratio_upper_bound_psr = sharpe_ratio_upper_bound * psr_set
    if len(sharpe_ratio_upper_bound_psr) > l:
        action_psr1 = np.argmax(sharpe_ratio_upper_bound_psr[:l])
        action_psr2 = np.argmax(sharpe_ratio_upper_bound_psr[l:]) + l
    else:
        action_psr1 = np.argmax(sharpe_ratio_upper_bound_psr)
        action_psr2 = None

    # Calcular theta_psr y los pesos de la estrategia PSR
    if action_psr1 is not None and action_psr2 is not None:
        theta_psr = portfolio_variances[action_psr1] / (portfolio_variances[action_psr1] + portfolio_variances[action_psr2])
        psr_weight = (1 - theta_psr) * eigenvectors_normalized[:, action_psr1] + theta_psr * eigenvectors_normalized[:, action_psr2]
    else:
        psr_weight = np.zeros(len(df_l.columns))  # Peso cero como valor predeterminado

    strategies = [optimal_weight, equal_weight, min_var_weight, psr_weight]
    beta_draws = [np.random.beta(success[i], fail[i]) for i in range(4)]
    selected_strategy = strategies[np.argmax(beta_draws)]

    # Paso 19: Calcular recompensa y actualizar contadores
    reward = selected_strategy.T @ df_portafolios.values
    rewards.append(reward)

    mv_reward = psr_weight.T @ df_portafolios.iloc[:, t].values
    mv_rewards.append(mv_reward)

    # Actualizar éxitos y fracasos
    strategy_rewards = [strategy.T @ df_portafolios.iloc[:, t].values for strategy in strategies]
    if np.max(strategy_rewards) == reward:
        success[np.argmax(beta_draws)] += 1
    else:
        fail[np.argmax(strategy_rewards)] += 1

# Convertir recompensas en un DataFrame para análisis
rewards_df = pd.DataFrame({
    't': range(len(rewards)),
    'rewards': rewards,
    'mv_rewards': mv_rewards
})

# Mostrar las primeras filas
print(rewards_df.head())


C:\Users\HP\AppData\Local\Temp\ipykernel_33148\1933718833.py:74: RuntimeWarning: invalid value encountered in sqrt
  denom = np.sqrt(np.abs(1 + 0.5 * sr**2 - skewness * sr + ((kurt - 3) / 4) * sr**2) / (n - 1))


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 200 is different from 24)

In [108]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from scipy.stats import skew, kurtosis
from scipy.stats import norm
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

class BanditPortfolio:
    def __init__(self, R):
        self.R = R
        self.n_arms, self.n_samples = R.shape
            
    def TS(self, window_size):
        self.reward = np.ones(self.n_samples - window_size)
        self.mv_reward = np.ones(self.n_samples - window_size)
        self.played_times = np.ones(self.n_arms)
        self.success = np.ones(4)
        self.fail = np.ones(4)

        for t in range(window_size, self.n_samples):

            sliceR = self.R[:, t-window_size:t]

            # Compute the covariance matrix
            covariance_matrix = np.cov(sliceR)  # histroical covariance

            # Eigenvalue Decomposition
            A, H = np.linalg.eig(covariance_matrix)  # equation 5

            # All eigenvalues are positive
            assert(np.sum(A < 0) == 0)

            # Sort the eigenvalues
            idx = np.argsort(-A)  # sort eigenvalues
            A = np.diag(A[idx])  #  eigenvalues as vector
            H = H[:, idx]  #  n(number of assets) orthonormal portfolios
            l = np.argwhere(np.median(np.diag(A)) > np.diag(A))[0][0]

            # Normalized weight
            H /= np.sum(H, axis=0)  # equation 7 normalized eigenvectors
            #  equation 8 normalized eigenvalues matrix
            ANew = H.T.dot(covariance_matrix).dot(H)

            # Compute the sharpe ratio
            #  Return of each port. in the slice window
            portfolio_reward = H.T.dot(sliceR)
            # Estimator of portfolio return, rolling average
            sharpe_ratio = np.mean(
                portfolio_reward, axis=1) / np.sqrt(ANew.diagonal())
            # Normalize the sharpe ratio
            #sharpe_ratio = MinMaxScaler().fit_transform(
            #    sharpe_ratio.reshape(-1, 1)).reshape(-1)

            # Compute the upper bound of expected reward
            sharpe_ratio_upper_bound = sharpe_ratio + \
                np.sqrt((2*np.log(t))/(window_size+self.played_times))

            # Select the optimal arm
            #  passive portfolios
            action1 = np.argmax(sharpe_ratio_upper_bound[:l])
            #  active portfolios
            action2 = np.argmax(sharpe_ratio_upper_bound[l:])+l

            #  update the times portfolio played
            self.played_times[action1] += 1
            self.played_times[action2] += 1  # update the second action

            # Optimal weight, Min var allocation between 2 chosen portfolios
            Adiag = ANew.diagonal()
            theta = Adiag[action1] / (Adiag[action1] + Adiag[action2])
            
            self.psr_set = []
            #psr_second_set = []
            for a in range(len(sharpe_ratio)):
                sr = sharpe_ratio[a]
                n = (window_size+self.played_times[a])/window_size
                skewness = skew(portfolio_reward[a, :])
                kurto = kurtosis(portfolio_reward[a, :])
                nomin = (sr-np.mean(sharpe_ratio))*np.sqrt(n)
                denom = np.sqrt(np.abs((1 + 0.5*sr**2 - skewness*sr + ((kurto-3)/4)*sr**2))/(n-1))
                self.psr_set.append(nomin/denom)
                
                
            self.psr_set = np.array([norm.cdf(a) for a in self.psr_set])
            
            sharpe_ratio_upper_bound_psr = (sharpe_ratio + \
                np.sqrt((2*np.log(t))/(window_size+self.played_times)))*np.array(self.psr_set)
                
            
            action_1_1 = np.argmax(sharpe_ratio_upper_bound_psr[:l])
            action_1_2 = np.argmax(sharpe_ratio_upper_bound_psr[l:])+l


            # Optimal weight
            theta_ = Adiag[action_1_1] / (Adiag[action_1_1] + Adiag[action_1_2])
                        
            
            
            self.weight_1 = (1-theta)*H[:, action1] + theta*H[:, action2]
            
            self.weight_2 = np.ones(self.n_arms)/self.n_arms
            
            self.weight_3 = (np.linalg.inv(covariance_matrix)@np.ones(self.n_arms).reshape(-1, 1))/(np.ones(
                self.n_arms).reshape(-1, 1).T@np.linalg.inv(covariance_matrix)@np.ones(self.n_arms).reshape(-1, 1))
            
            self.weight_4 = (1-theta_)*H[:,action_1_1] + theta_*H[:,action_1_2]
            
            
            final_actions = [self.weight_1, self.weight_2, self.weight_3, self.weight_4]
            draws = [np.random.beta(self.success[action], self.fail[action])
                     for action in range(4)]
            
            final_weight = final_actions[np.argmax(draws)]
            
            self.reward[t-window_size] = final_weight.T.dot(self.R[:, t])
            self.mv_reward[t-window_size] = self.weight_3.T.dot(self.R[:, t])
            
            rewards = [self.weight_1.T.dot(self.R[:, t]), self.weight_2.T.dot(self.R[:, t]), self.weight_3.T.dot(self.R[:, t]), self.weight_4.T.dot(self.R[:, t])]
            
            

            if np.max(rewards) == final_weight.T@self.R[:, t]:
                self.success[np.argmax(draws)] += 1
                other_idx = [a for a in range(4) if a != np.argmax(rewards)]
                self.fail[other_idx[0]] +=1
                self.fail[other_idx[1]] +=1
                self.fail[other_idx[2]] +=1
            else:
                other_idx = [a for a in range(4) if (a != np.argmax(rewards)) & (a != np.argmax(draws))]
                self.success[np.argmax(rewards)] +=1
                self.fail[other_idx[0]] +=1
                self.fail[other_idx[1]] +=1
                self.fail[np.argmax(draws)] += 1

In [111]:
R=df_portafolios
R[R < -99]
R = (R + 100) / 100  # Gross Return

window_size = 12
orthogonal_bandit_portfolio = BanditPortfolio(R)

In [112]:
orthogonal_bandit_portfolio.TS(window_size = window_size)
ts_ret = orthogonal_bandit_portfolio.reward
mv_ret = orthogonal_bandit_portfolio.mv_reward

InvalidIndexError: (slice(None, None, None), slice(0, 12, None))